In [1]:
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
library(parallel)

Warning message:
“package ‘plyr’ was built under R version 4.3.3”
Warning message:
“package ‘dplyr’ was built under R version 4.3.3”
Warning message:
“package ‘tidyr’ was built under R version 4.3.3”
Warning message:
“package ‘tibble’ was built under R version 4.3.3”


In [2]:
meta_data=read.csv("/home/workspace/IHA_Figure_Revision/Dataset/scRNA_meta_data-2024-05-09.csv")
meta_data_subset<-meta_data %>% 
filter(Covid_exclusion=='no',sample.daysSinceFirstVisit<600) %>%
filter(!sample.visitName%in% c('Flu Year 1 Day 7',
                              'Flu Year 2 Day 7'))

In [3]:
df_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
df<-read.csv(paste0("/home/workspace/IHA_Figure_Revision/Figure5/01_NMF_CD4_T_cell_Projection/NMF_Score_BRI/",x,".csv"))
df$pbmc_sample_id=x
return(df)
},mc.cores=60)

In [4]:
df_combined<-do.call(rbind,df_list)
df_combined_longformat<- pivot_longer(df_combined, cols = starts_with("NMF"), names_to = "NMF_Scores", values_to = "value")

In [5]:
df_combined_longformat<-df_combined_longformat %>% group_by(pbmc_sample_id,NMF_Scores,AIFI_L3) %>% summarise(AverageScore=mean(value))

`summarise()` has grouped output by 'pbmc_sample_id', 'NMF_Scores'. You can
override using the `.groups` argument.


In [6]:
unique(df_combined_longformat$NMF_Scores)

[1] "NMF0_Cytotoxic"  "NMF10_Tissue"    "NMF11_Th1"       "NMF1_Treg"      
 [5] "NMF2_Th17"       "NMF3_Naive"      "NMF4_Act"        "NMF5_Th2"       
 [9] "NMF6_Tfh"        "NMF7_IFN"        "NMF8_Cent_Mem"   "NMF9_Thymic_Emi"

In [7]:
df_combined_longformat_filtered<-df_combined_longformat %>% filter(NMF_Scores %in% c('NMF5_Th2',"NMF11_Th1",'NMF6_Tfh')) %>% 
filter(AIFI_L3 %in% c('GZMB- CD27+ EM CD4 T cell'))

In [8]:
df<-left_join(df_combined_longformat_filtered,meta_data_subset)

Joining with `by = join_by(pbmc_sample_id)`


In [9]:
write.csv(df,'Th2_Th1_Tfh_Baseline_Samples_BRI_GZMB-CD27+EMCD4Tcell.csv')